# AirBNB homes in Seattle and Boston.

In this notebook, I will answer three questions below:

1. What is the important element for hosts/rooms to be booked maltiple times?
2. Waht is the price range of hosts/rooms booked maltiple times?
3. What are the hosts/rooms' features with higher review score?

In [1]:
# import libralies here

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler as sc

%matplotlib inline

In [2]:
pd.set_option('display.max_columns', 100)

In [3]:
# read csv files that wil be used for nswering questions

seattle  = pd.read_csv('seattle_listtings.csv')
boston = pd.read_csv('boston_listings.csv')

Source of the data sets ^: http://insideairbnb.com/get-the-data.html

In [4]:
# checking the length of each data frame in case
print(len(seattle))
print(len(boston))

7017
3440


## 1. Data cleaning
- Combine dataframes of house in Seattle and on Boston in 2020.
- Making data frame of hosts who offered their place over 2 times in 2020 to answer question 1 and 2.

In [5]:
# checking if the dataframs have the same columns
seattle.columns == boston.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [32]:
# Combining the data frames
master_df = pd.concat([seattle, boston])
# Changing the name of id to listing_id
master_df.rename(columns={'id':'listing_id'}, inplace = True)
#Checking the lenght in case.
master_df.shape[0]

10457

In [7]:
#Making data set of host who offered their room multiple times (over 2)

popular_host = master_df.groupby('host_id').count()
popular_host = popular_host.loc[popular_host["listing_id"] >1]
popular_host.rename(columns = {"listing_id":"Count"}, inplace = True)
popular_host = popular_host[["Count", "listing_url"]]
popular_host = popular_host.merge(master_df, how= "left", on = "host_id")
popular_host.drop(['listing_url_x'], axis=1, inplace = True)

popular_host.head()

,host_id,Count,listing_id,listing_url_y,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,...,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2536,2,2318,https://www.airbnb.com/rooms/2318,20200521035945,2020-05-21,Casa Madrona - Urban Oasis 1 block from the park!,"Gorgeous, architect remodeled, Dutch Colonial ...","This beautiful, gracious home has been complet...","Gorgeous, architect remodeled, Dutch Colonial ...",none,Madrona is a hidden gem of a neighborhood. It ...,"We adhere to a 10pm -9am quiet hour schedule, ...",NaN,Guests can access any part of the house.,We are a family who live next door and are ava...,NaN,NaN,NaN,https://a0.muscache.com/im/pictures/02973ad3-a...,NaN,https://www.airbnb.com/users/show/2536,Megan,2008-08-26,"Seattle, Washington, United States",I welcome guests from all walks of life and ev...,within a few hours,67%,88%,t,https://a0.muscache.com/im/pictures/user/016a1...,https://a0.muscache.com/im/pictures/user/016a1...,Minor,2.0,2.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,f,"Seattle, WA, United States",Madrona,Madrona,Central Area,Seattle,WA,98122,Seattle,"Seattle, WA",US,United States,47.61082,...,4.0,Real Bed,"{Internet,Wifi,Kitchen,""Free parking on premis...",NaN,$296.00,NaN,NaN,$500.00,$250.00,8,$25.00,1,1125,1,1,1125,1125,1.0,1125.0,today,t,29,59,89,132,2020-05-21,32,7,2008-09-15,2020-02-01,100.0,10.0,10.0,10.0,10.0,10.0,10.0,t,STR-OPLI-19-002837,"{WASHINGTON,"" Seattle"","" WA""}",t,f,strict_14_with_grace_period,f,f,2,2,0,0,0.22
1,2536,2,30013204,https://www.airbnb.com/rooms/30013204,20200521035945,2020-05-21,Casita Madrona Annex,"Welcome to Casita Madrona Annex, a 1+ bedroom,...",Casita Madrona has a private entrance through ...,"Welcome to Casita Madrona Annex, a 1+ bedroom,...",none,NaN,The Casita is cute and slightly quirky in that...,NaN,NaN,NaN,Quiet hours start at 8:30 PM. This is an oasis...,NaN,NaN,https://a0.muscache.com/im/pictures/32285d08-8...,NaN,https://www.airbnb.com/users/show/2536,Megan,2008-08-26,"Seattle, Washington, United States",I welcome guests from all walks of life and ev...,within a few hours,67%,88%,t,https://a0.muscache.com/im/pictures/user/016a1...,https://a0.muscache.com/im/pictures/user/016a1...,Minor,2.0,2.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,f,"Seattle, WA, United States",Madrona,Madrona,Central Area,Seattle,WA,98122,Seattle,"Seattle, WA",US,United States,47.61099,...,1.0,Real Bed,"{Wifi,""Free parking on premises"",Heating,

## 2. Data Analysis

### 2.1. Analyzing features of hosts who offered their place over two times by using clusterinig

In this section, I will use clustering to find the features of hosts who offered their room to guests over 2 times to answer the question 1 and 2.

The elements of this analysis are: 
1. review_scores_rating
2. host_response_rate
3. price

In [8]:
popular_host_clusterning = popular_host[['host_id','review_scores_rating', 'host_response_rate', 'price']].set_index('host_id')
popular_host_clusterning.dropna(inplace = True)
popular_host_clusterning['host_response_rate'] = popular_host_clusterning['host_response_rate'].str.replace('%', '').astype('float')
popular_host_clusterning['price'] = popular_host_clusterning['price'].str.replace('$', '')
popular_host_clusterning['price'] = popular_host_clusterning['price'].str.replace(',', '').astype('float')
popular_host_clusterning.shape

(4419, 3)

In [9]:
popular_host_clusterning.columns

Index(['review_scores_rating', 'host_response_rate', 'price'], dtype='object')

In [10]:
popular_host_clusterning = popular_host_clusterning.groupby('host_id').agg(
    {'review_scores_rating': 'mean',
     'host_response_rate': 'mean',
     'price': 'mean'
    })

In [11]:
popular_host_clusterning.dtypes

review_scores_rating    float64
host_response_rate      float64
price                   float64
dtype: object

In [12]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
popular_host_clusterning_sc = sc.fit_transform(popular_host_clusterning)

In [31]:
kmeans = KMeans(n_clusters=6, random_state=0)
clusters = kmeans.fit(popular_host_clusterning_sc)
popular_host_clusterning["cluster"] = clusters.labels_
print(popular_host_clusterning["cluster"].unique())
popular_host_clusterning.head()

[5 4 0 1 3 2]


,review_scores_rating,host_response_rate,price,cluster
host_id,,,,
2536,96.500000,67.0,200.500000,5
4193,95.666667,100.0,35.000000,4
7969,89.500000,83.0,25.500000,0
8229,95.000000,100.0,187.666667,4
14942,93.000000,100.0,169.666667,0


In [14]:
popular_host_clusterning.groupby("cluster").count()

,review_scores_rating,host_response_rate,price
cluster,,,
0,199,199,199
1,16,16,16
2,1,1,1
3,25,25,25
4,671,671,671
5,60,60,60


In [15]:
popular_host_clusterning.groupby("cluster").mean()

,review_scores_rating,host_response_rate,price
cluster,,,
0,90.193588,98.115063,123.672207
1,91.121795,20.625000,188.901442
2,85.500000,97.000000,10000.000000
3,73.753756,91.400000,225.595855
4,97.427703,99.323398,150.147864
5,95.348731,72.183333,151.465393


### FInding 2.1.

We can devide the popular hosts into 6 groups:

0. high review score (90), high response rate (98), average price 123 - Total(199)
1. high review score (91), very low response rate (21), average price 188 - Total(16)
2. Exceptioanl case, very high price 10000 - Only 1
3. low review score (73), high response rate (91), average price 226 - Total(25)
4. highest review score (98), highest response rate (99), average price 150 - Total(671)
5. high review score (95), low response rate (72), average price 152 - Total(60)

Group 4 is the most poupar group in all, and its features are "highest review score (98), highest response rate (99), average price 150"
The second biggest destribution of them is occupied by group 0 with the features of high review score (90), high response rate (98), average price 123. 

Judging from the group 0 and 4, popular hosts tend to response to most messages and get high review. Also, their room price is between 123 and 150. 

### 2.2. Analyze the status of cancelation policy of high rated place.
Next, I will analyze the elements causing higher review scores.

In [43]:
# Make a dataframe of places whose review_scores_rating is over  95
high = master_df[master_df['review_scores_rating'] >= 95]

In [47]:
high.cancellation_policy.unique()

array(['strict_14_with_grace_period', 'moderate', 'flexible',
       'super_strict_60', 'super_strict_30', 'strict'], dtype=object)

In [51]:
# relationshio between cancelation policy and rating
cancelation = ['strict_14_with_grace_period', 'moderate', 'flexible', 'super_strict_60', 'super_strict_30', 'strict']
dist_of_cancelation = []

for cancel in cancelation:
    length = len(high[high['cancellation_policy'] == cancel])
    dist_of_cancelation.append(length)


#calculate the proportion of each cancelaton policy (strict = 0, moderate = 1, flexible = 2) 
total = len(high)

print('strict_14_with_grace_period: ' + str(dist_of_cancelation[0]/total))
print('moderate: ' + str(dist_of_cancelation[1]/total))
print('flexible: ' + str(dist_of_cancelation[2]/total))
print('super_strict_60: ' + str(dist_of_cancelation[3]/total))
print('super_strict_30: ' + str(dist_of_cancelation[4]/total))
print('strict: ' + str(dist_of_cancelation[5]/total))

strict_14_with_grace_period: 0.3729845343863113
moderate: 0.3904244817374136
flexible: 0.22639025995393222
super_strict_60: 0.004442250740375123
super_strict_30: 0.0042777229351760445
strict: 0.0014807502467917078


### Finding 2.2.
In the places with higher rate, the cancelation policy tends to be modern or strict. <br>
There are three main components: "strict_14_with_grace_period", "moderate" and  "flexible." <br>
The percentage of flexible policy is lower (22%) than other two (over 35%) 

### 2.3 Analyze the importance of host_response_rate for getting higher rate

In [56]:
high_response = high.dropna(subset = ['host_response_rate'])

high_response['host_response_rate_flag'] = 0
high_response['host_response_rate'] = high_response['host_response_rate'].str.replace('%', '').astype('int')

high_response['host_response_rate_flag'] = high_response['host_response_rate_flag'].where(high_response['host_response_rate'] < 90, 1)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [57]:
high_response['host_response_rate_flag']

0       0
3       1
4       1
6       1
7       1
       ..
3313    1
3323    1
3372    1
3409    1
3412    1
Name: host_response_rate_flag, Length: 4613, dtype: int64

In [58]:
high_response['host_response_rate_flag'].sum()/len(high_response)

0.9308476045957078

### Finding 2.3.
About 90% of hosts with higher review scores has over 90% of response rate. 

## 2.4. Analyze the popular room type
Finally, I will analyze what is popular room type. 

In [59]:
# relationshio between cancelation policy and rating
rooms = ['Entire home/apt', 'Private room', 'Shared room']
dist_of_cancelation = []

for room in rooms:
    length = len(high[high['room_type'] == room])
    dist_of_cancelation.append(length)


#calculate the proportion of each cancelaton policy (strict = 0, moderate = 1, flexible = 2) 
total = len(high)

print('Entire home/apt: ' + str(dist_of_cancelation[0]/total))
print('Private room: ' + str(dist_of_cancelation[1]/total))
print('Shared room: ' + str(dist_of_cancelation[2]/total))

Entire home/apt: 0.732313260941099
Private room: 0.2548535702533728
Shared room: 0.007568279039157618


### 2.4. Finding 
It shows that over 70 of room type of rooms with high review score is "Entire home/apt."
Surprisingly, shared room occupied only 0.008%.

## Answering Question

### 1. What are the important elements for hosts/rooms to be booked maltiple times?
Answer - Judging from the result of clustering, review score and response rate are the important elements and they should be high (over 90%)

### 2. What is the price range of hosts/rooms booked maltiple times?
Answer - The average price of 870 hosts/rooms is between $ 123 and ＄150 out of 972 hosts/rooms.  

### 3. What are the features of hosts/rooms with higher review score? 
Answer - It can be seen that the hosts/rooms with higher review scores have high response rate (93%).<br>
Additionally, 73% of roomtypes of them is  Entire home/apt


